In [23]:
def read_file(day, ex=1):
    with open(f"inputs/day{day}-{ex}.txt", "r") as f:
        return f.readlines()

In [24]:
day=1
ex=1

last_number = None
increases = 0
for number in read_file(day, ex):
    if last_number and int(number) > last_number:
        increases += 1
    last_number = int(number)
print(increases)

1466


In [25]:
day=1
ex=2

window = [0, 0, 0]
increases = 0
for i, number in enumerate(read_file(day, 1)): # same input as 1
    
    if i >= 3 and int(number) > window[i % 3]:
        increases += 1
    window[i % 3] = int(number)
print(increases)

1491


In [26]:
day=2
ex=1
from operator import add, mul
from functools import reduce, partial

def parse_command(cmd: str):
    axis, unit = cmd.split()
    ax = {
        'forward': [0, 1, ],
        'up': [-1, 0], 
        'down': [1, 0]
    }

    return list(map(mul, ax[axis], [int(unit)]*2))

pos = list(reduce( partial(map, add), map(parse_command, read_file(day, ex)), [0,0]))
print(mul(*pos))


1499229


In [27]:
day=2
ex=2

def parse_command_2(cmd: str):
    axis, unit = cmd.split()
    un = int(unit)
    ax = {
        'forward': [0, 1, 0],
        'up': [0, 0, -1], 
        'down': [0, 0, 1]
    }

    return list(map(mul, ax[axis], [int(unit)]*3))

def red(a, b):
    return [a[0]+a[2]*b[1], a[1]+b[1], a[2]+b[2]]

pos = list(reduce(red, map(parse_command_2, read_file(day, 1)), [0,0,0]))
print(pos[0]*pos[1])


1340836560


In [28]:
!pip install bitarray

You should consider upgrading via the '/Users/w1ukxd2/.venv/aoc/bin/python -m pip install --upgrade pip' command.


In [29]:
day=3
ex=1

from bitarray import bitarray, util
count = 0
sum_a = []
for line in read_file(day, 1):
    count += 1
    line = line.replace('\n', '')
    if not sum_a:
        n = len(line)
        sum_a = [0]*n
    sum_a = list(map(add, sum_a, map(int, line)))


bits = list(map(lambda x: x > (count / 2), sum_a))
gamma = util.ba2int(bitarray(bits))
eps = util.ba2int(~bitarray(bits))

print(gamma*eps)



2967914


In [30]:
day=3
ex=2

f_tree = {'0': [], '1': []}
for line in read_file(day, 1):
    f_tree[line[0]].append(line.replace('\n', ''))


def calculate_o2(arr, i, most):
    tree = {'0': [], '1': []}
    if len(arr) == 1:
        return util.ba2int(bitarray(arr[0]))

    for line in arr:
        tree[line[i]].append(line.replace('\n', ''))

    sor = sorted(tree.values(), key=len)
    return calculate_o2(sor[most], i+1, most)

sor = sorted(f_tree.values(), key=len)
o2 = calculate_o2(sor[-1], 1, -1)
co2 = calculate_o2(sor[0], 1, 0)

print(o2*co2)
    

7041258


In [31]:
day=4
ex=1

class Bingo:
    def __init__(self) -> None:
        self.rows = []
        self.drawn = []

    def __repr__(self) -> str:
        return f"""
        rows: {self.rows}
        drawn: {self.drawn}"""

    def draw(self, n):
        if n in self.rows:
            self.drawn.append(n)
            for i in range(0, 25, 5):
                # line
                r = set(self.rows[i:(i+5)]).issubset(set(self.drawn))
                if r:
                    break
            for i in range(5):
                # columns
                cols = set(map(self.rows.__getitem__, range(i, 25+i, 5)))
                c = cols.issubset(set(self.drawn))
                if c:
                    break
        
            if r or c:
                return sum(self.rows) - sum(self.drawn)

def populate_bingos():
    random_numbers = list()
    bingo = None
    bingos = []
    for i, line in enumerate(read_file(day ,1)):
        if i == 0:
            ns = line.split(',')
            random_numbers = list(map(int, ns))
        elif line == '\n':
            if bingo:
                bingos.append(bingo)
            bingo = Bingo()
            continue
        else:
            row = list(map(int, line.split()))
            bingo.rows.extend(row)

    if bingos[-1] != bingo:
        bingos.append(bingo)
    return random_numbers, bingos
    

def find_winners():
    random_numbers, bingos = populate_bingos()
    for n in random_numbers:
        i = 0
        while i < len(bingos):
            b = bingos[i]
            complete = b.draw(n)
            if complete:
                yield complete*n
                del bingos[i]
            else:
                i +=1

print(next(find_winners())) # 41503
                

41503


In [32]:
day=4
ex=2

c = list(find_winners())
print(c[-1])



3178


In [33]:
day=5
ex=1
import math


def parse_coord(c: str):
    return tuple(map(int, c.split(',')))
def parse_line_5(line: str):
    xy1, _, xy2 = line.split()
    return parse_coord(xy1), parse_coord(xy2)

sign = lambda x: int(math.copysign(1, x))
def populate_points(with_diag=False):
    for line in read_file(day, 1):
        xy1, xy2 = parse_line_5(line)
        diff_x = xy2[0] - xy1[0]
        dir_x = sign(diff_x)
        diff_y = xy2[1] - xy1[1]
        dir_y = sign(diff_y)
        if diff_x != 0 and diff_y != 0:
            if not with_diag:
                continue
            m = int(diff_y / diff_x)
            for i in range(xy1[0], xy2[0]+dir_x, int(dir_x)):
                yield (i, m*(i - xy1[0]) + xy1[1])
        elif diff_x != 0:
            for i in range(xy1[0], xy2[0]+dir_x, int(dir_x)):
                p = (i, xy1[1])
                yield p
        elif diff_y != 0:
            for i in range(xy1[1], xy2[1]+dir_y, int(dir_y)):
                p = (xy1[0], i)
                yield p

def count_overlaps(wiht_diag=False):
    points = {}
    two = 0
    for p in populate_points(wiht_diag):
        points[p] = points.get(p, 0) + 1
        if points[p] == 2:
            two += 1
    return two


# print(points)
print(count_overlaps())



8622


In [34]:
day=5
ex=2

print(count_overlaps(True))


22037


In [50]:
day=6
ex=1

def calculate(days):
    initial = list(map(int, read_file(day, 1)[0].split(',')))
    individuals = [0]*9
    for i in initial:
        individuals[i] += 1

    for i in range(days):
        zeros = individuals[0]
        individuals = individuals[1:] + [0]
        individuals[8] = zeros
        individuals[6] += zeros

    print(sum(individuals))

calculate(80)


388739


In [51]:
day=6
ex=2

calculate(256)

1741362314973
